In [1]:
import sddk
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
import getpass
from github import Github
import requests
import zipfile
import io

In [2]:
g = Github("kasev", getpass.getpass("github password: "))

In [3]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/


In [4]:
LAGT = s.read_file("SDAM_data/AGT/AGT_raw_20240107.json")

In [5]:
len(LAGT)

1892

In [6]:
LAGT = LAGT.drop_duplicates(subset='doc_id', keep='last')
len(LAGT)

1818

In [7]:
LAGT.head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source
0,ggm0001,ggm0001.ggm001,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr
3,stoa0033a,stoa0033a.tlg028,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr
4,stoa0033a,stoa0033a.tlg043,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr
5,stoa0121,stoa0121.stoa001,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr


In [8]:
LAGT["lemmatized_sentences"] = None

# MorphGNT

In [9]:
resp = requests.get("https://github.com/morphgnt/sblgnt")

In [10]:
repo = g.get_user("morphgnt").get_repo("sblgnt" )
contents = repo.get_contents("")
filenames = [content.path for content in contents if ".txt" in content.path]

In [11]:
len(filenames)

27

In [12]:
base_url = "https://raw.githubusercontent.com/morphgnt/sblgnt/master/"
# test with individual book
book = pd.read_csv(base_url + filenames[0], delimiter=" ", header=None)

In [13]:
book.head(10)

,0,1,2,3,4,5,6
0,10101,N-,----NSF-,Βίβλος,Βίβλος,βίβλος,βίβλος
1,10101,N-,----GSF-,γενέσεως,γενέσεως,γενέσεως,γένεσις
2,10101,N-,----GSM-,Ἰησοῦ,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς
3,10101,N-,----GSM-,χριστοῦ,χριστοῦ,Χριστοῦ,Χριστός
4,10101,N-,----GSM-,υἱοῦ,υἱοῦ,υἱοῦ,υἱός
5,10101,N-,----GSM-,Δαυὶδ,Δαυὶδ,Δαυίδ,Δαυίδ
6,10101,N-,----GSM-,υἱοῦ,υἱοῦ,υἱοῦ,υἱός
7,10101,N-,----GSM-,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,Ἀβραάμ
8,10102,N-,----NSM-,Ἀβραὰμ,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ
9,10102,V-,3AAI-S--,ἐγέννησεν,ἐγέννησεν,ἐγέννησε(ν),γεννάω


In [14]:
book = pd.read_csv(base_url + filenames[0], delimiter=" ", header=None)
book.columns = ["verse", "pos", "parsing", "word_punct", "word", "norm", "lemma"]
book_sents = []
sent_lemmata = []
raw_sent = ""
for word, pos, lemma in zip(book["word_punct"], book["pos"], book["lemma"]):
    raw_sent += word + " "
    if pos[0] in ["N", "A", "V"]:
        sent_lemmata.append(lemma)
    if re.search("[\·\.\;\!]", word[-1]):
        book_sents.append(sent_lemmata) # (raw_sent, sent_lemmata))
        sent_lemmata = []
        raw_sent = ""

In [15]:
len(book_sents)

1679

In [16]:
books_sents = []
for filename in filenames:
    book = pd.read_csv(base_url + filename, delimiter=" ", header=None)
    book.columns = ["verse", "pos", "parsing", "word_punct", "word", "norm", "lemma"]
    book_sents = [] 
    sent_lemmata = []
    raw_sent = ""
    for word, pos, lemma in zip(book["word_punct"], book["pos"], book["lemma"]):
        raw_sent += word + " "
        if pos[0] in ["N", "A", "V"]:
            sent_lemmata.append(lemma)
        if re.search("[\·\.\;\!]", word[-1]):
            book_sents.append(sent_lemmata) # (raw_sent, sent_lemmata))
            sent_lemmata = []
            raw_sent = ""
    books_sents.append({"filename": filename, "lemmatized_sentences" : book_sents})

In [17]:
len(books_sents)

27

In [18]:
morphgnt_dict = dict(zip(LAGT[LAGT["doc_id"].str.startswith("tlg0031")]["doc_id"].tolist(), books_sents))

In [19]:
# this one will be gradually filled in by tlg ids and the lemmata sources
lemmata_source_dict = {}

In [20]:
def get_from_morphgnt(doc_id, lemmatized_sentences):
    if doc_id in morphgnt_dict.keys():
        lemmatized_sentences = morphgnt_dict[doc_id]["lemmatized_sentences"]
        lemmata_source_dict[doc_id] = "morphgnt"
    return lemmatized_sentences

LAGT["lemmatized_sentences"] = LAGT.apply(lambda row: get_from_morphgnt(row["doc_id"], row["lemmatized_sentences"]), axis=1)

In [21]:
LAGT[LAGT["doc_id"].str.startswith("tlg0031")].head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences
1525,tlg0031,tlg0031.tlg001,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew,ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱ...,18288,perseus,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
1526,tlg0031,tlg0031.tlg002,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark,\n\n ΑΡΧΗ τοῦ εὐαγγελίου Ἰη...,11275,perseus,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ..."
1527,tlg0031,tlg0031.tlg003,tlg0031.tlg003.perseus-grc2.xml,None,New Testament - Luke,\n\n ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχε...,19459,perseus,"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη..."
1528,tlg0031,tlg0031.tlg004,tlg0031.tlg004.perseus-grc2.xml,None,New Testament - John,"\n\n ΕΝ ΑΡΧΗ ἦν ὁ λόγος, κα...",15592,perseus,"[[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ..."
1529,tlg0031,tlg0031.tlg005,tlg0031.tlg005.perseus-grc2.xml,None,New Testament - Acts,\n τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων...,18405,perseus,"[[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰ..."


In [22]:
lemmata_source_dict

{'tlg0031.tlg001': 'morphgnt',
 'tlg0031.tlg002': 'morphgnt',
 'tlg0031.tlg003': 'morphgnt',
 'tlg0031.tlg004': 'morphgnt',
 'tlg0031.tlg005': 'morphgnt',
 'tlg0031.tlg006': 'morphgnt',
 'tlg0031.tlg007': 'morphgnt',
 'tlg0031.tlg008': 'morphgnt',
 'tlg0031.tlg009': 'morphgnt',
 'tlg0031.tlg010': 'morphgnt',
 'tlg0031.tlg011': 'morphgnt',
 'tlg0031.tlg012': 'morphgnt',
 'tlg0031.tlg013': 'morphgnt',
 'tlg0031.tlg014': 'morphgnt',
 'tlg0031.tlg015': 'morphgnt',
 'tlg0031.tlg016': 'morphgnt',
 'tlg0031.tlg017': 'morphgnt',
 'tlg0031.tlg018': 'morphgnt',
 'tlg0031.tlg019': 'morphgnt',
 'tlg0031.tlg020': 'morphgnt',
 'tlg0031.tlg021': 'morphgnt',
 'tlg0031.tlg022': 'morphgnt',
 'tlg0031.tlg023': 'morphgnt',
 'tlg0031.tlg024': 'morphgnt',
 'tlg0031.tlg025': 'morphgnt',
 'tlg0031.tlg026': 'morphgnt',
 'tlg0031.tlg027': 'morphgnt'}

# LXX

In [23]:
LAGT[LAGT["author"].str.contains("Septuaginta", na=False)].head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences
347,tlg0527,tlg0527.tlg001,tlg0527.tlg001.opp-grc2.xml,Septuaginta,Genesis,ΕΝ APXH ἐποίησεν ὁ θεὸς τὸν οὐρανὸν καὶ τὴν γῆ...,46422,1Kgr,None
348,tlg0527,tlg0527.tlg002,tlg0527.tlg002.opp-grc2.xml,Septuaginta,Exodus,ταῦτα τὰ ὀνόματα τῶν υἱῶν Ἰσραὴλ τῶν εἰσπεπορε...,36962,1Kgr,None
349,tlg0527,tlg0527.tlg003,tlg0527.tlg003.opp-grc2.xml,Septuaginta,Leviticus,KAI ἀνεκάλεσεν Μωυσῆν καὶ ἐλάλησεν Κύριος αὐτῷ...,27730,1Kgr,None
350,tlg0527,tlg0527.tlg004,tlg0527.tlg004.opp-grc2.xml,Septuaginta,Numeri,KAI ἐλάλησεν κύριος πρὸς Μωυσῆν ἐν τῇ ἐρήμῳ τῇ...,36189,1Kgr,None
351,tlg0527,tlg0527.tlg005,tlg0527.tlg005.opp-grc2.xml,Septuaginta,Deuteronomium,OYTOI οἱ λόγοι οὓς ἐλάλησεν Μωυσῆς παντὶ Ἰσραὴ...,33651,1Kgr,None


In [24]:
repo = g.get_user("nathans").get_repo("lxxmorph-unicode")
contents = repo.get_contents("")
filenames = [content.path for content in contents if re.search("^\d\d\.", content.path)]
filenames[:10]

['01.Gen.txt',
 '02.Exod.txt',
 '03.Lev.txt',
 '04.Num.txt',
 '05.Deut.txt',
 '06.JoshB.txt',
 '07.JoshA.txt',
 '08.JudgesB.txt',
 '09.JudgesA.txt',
 '10.Ruth.txt']

In [25]:
base_url = "https://raw.githubusercontent.com/nathans/lxxmorph-unicode/master/"
lines = requests.get(base_url + filenames[0]).text.split("\n")

In [26]:
lines[:15]

['Gen 1:1',
 'ἐν P--------- ἐν',
 'ἀρχῇ N1--DSF--- ἀρχή',
 'ἐποίησεν VAI-AAI3S- ποιέω',
 'ὁ RA--NSM--- ὁ',
 'θεὸς N2--NSM--- θεός',
 'τὸν RA--ASM--- ὁ',
 'οὐρανὸν N2--ASM--- οὐρανός',
 'καὶ C--------- καί',
 'τὴν RA--ASF--- ὁ',
 'γῆν N1--ASF--- γῆ',
 '',
 'Gen 1:2',
 'ἡ RA--NSF--- ὁ',
 'δὲ X--------- δέ']

In [27]:
book_verses = []
verse = []
for line in lines:
    line_split = line.split()
    if len(line_split) == 3:
        if line_split[1][0] in ["N", "V", "A"]:
            verse.append(line_split[2])
    if len(line_split) == 0:
        book_verses.append(verse)
        verse= []

In [28]:
book_verses[:3]

[['ἀρχή', 'ποιέω', 'θεός', 'οὐρανός', 'γῆ'],
 ['γῆ',
  'εἰμί',
  'ἀόρατος',
  'ἀκατασκεύαστος',
  'σκότος',
  'ἄβυσσος',
  'πνεῦμα',
  'θεός',
  'ὕδωρ'],
 ['εἶπον', 'θεός', 'γίγνομαι', 'φῶς', 'γίγνομαι', 'φῶς']]

In [29]:
base_url = "https://raw.githubusercontent.com/nathans/lxxmorph-unicode/master/"

books_sents = {}
for filename in filenames:
    lines = requests.get(base_url + filename).text.split("\n")
    book_verses = []
    verse = []
    for line in lines:
        line_split = line.split()
        if len(line_split) == 3:
            if line_split[1][0] in ["N", "V", "A"]:
                verse.append(line_split[2])
        if len(line_split) == 0:
            book_verses.append(verse)
            verse= []
    books_sents[filename] = book_verses

In [30]:
filenames

['01.Gen.txt',
 '02.Exod.txt',
 '03.Lev.txt',
 '04.Num.txt',
 '05.Deut.txt',
 '06.JoshB.txt',
 '07.JoshA.txt',
 '08.JudgesB.txt',
 '09.JudgesA.txt',
 '10.Ruth.txt',
 '11.1Sam.txt',
 '12.2Sam.txt',
 '13.1Kings.txt',
 '14.2Kings.txt',
 '15.1Chron.txt',
 '16.2Chron.txt',
 '17.1Esdras.txt',
 '18.2Esdras.txt',
 '19.Esther.txt',
 '20.Judith.txt',
 '21.TobitBA.txt',
 '22.TobitS.txt',
 '23.1Macc.txt',
 '24.2Macc.txt',
 '25.3Macc.txt',
 '26.4Macc.txt',
 '27.Psalms.txt',
 '28.Odes.txt',
 '29.Proverbs.txt',
 '30.Qoheleth.txt',
 '31.Canticles.txt',
 '32.Job.txt',
 '33.Wisdom.txt',
 '34.Sirach.txt',
 '35.PsSol.txt',
 '36.Hosea.txt',
 '37.Micah.txt',
 '38.Amos.txt',
 '39.Joel.txt',
 '40.Jonah.txt',
 '41.Obadiah.txt',
 '42.Nahum.txt',
 '43.Habakkuk.txt',
 '44.Zeph.txt',
 '45.Haggai.txt',
 '46.Zech.txt',
 '47.Malachi.txt',
 '48.Isaiah.txt',
 '49.Jer.txt',
 '50.Baruch.txt',
 '51.EpJer.txt',
 '52.Lam.txt',
 '53.Ezek.txt',
 '54.BelOG.txt',
 '55.BelTh.txt',
 '56.DanielOG.txt',
 '57.DanielTh.txt',
 '58.Sus

In [31]:
#pd.DataFrame((zip(LAGT[LAGT["author_id"]=="tlg0527"]["doc_id"], LAGT[LAGT["author_id"]=="tlg0527"]["title"], filenames))).to_csv("../data/lxx_mapping.csv", index=False)

In [32]:
# load back the manually mapped filenames
lxx_mapping_dict = pd.read_csv("../data/lxx_mapping.csv", usecols=[0,2], index_col=0, header=None)[2].to_dict()
lxx_mapping_dict

{'tlg0527.tlg001': '01.Gen.txt',
 'tlg0527.tlg002': '02.Exod.txt',
 'tlg0527.tlg003': '03.Lev.txt',
 'tlg0527.tlg004': '04.Num.txt',
 'tlg0527.tlg005': '05.Deut.txt',
 'tlg0527.tlg006': '06.JoshB.txt',
 'tlg0527.tlg008': '08.JudgesB.txt',
 'tlg0527.tlg010': '10.Ruth.txt',
 'tlg0527.tlg011': '11.1Sam.txt',
 'tlg0527.tlg012': '12.2Sam.txt',
 'tlg0527.tlg013': '13.1Kings.txt',
 'tlg0527.tlg014': '14.2Kings.txt',
 'tlg0527.tlg015': '15.1Chron.txt',
 'tlg0527.tlg016': '16.2Chron.txt',
 'tlg0527.tlg017': '17.1Esdras.txt',
 'tlg0527.tlg018': '18.2Esdras.txt',
 'tlg0527.tlg019': '19.Esther.txt',
 'tlg0527.tlg020': '20.Judith.txt',
 'tlg0527.tlg021': '22.TobitS.txt',
 'tlg0527.tlg023': '23.1Macc.txt',
 'tlg0527.tlg024': '24.2Macc.txt',
 'tlg0527.tlg025': '25.3Macc.txt',
 'tlg0527.tlg026': '26.4Macc.txt',
 'tlg0527.tlg027': '27.Psalms.txt',
 'tlg0527.tlg028': '28.Odes.txt',
 'tlg0527.tlg029': '29.Proverbs.txt',
 'tlg0527.tlg031': '31.Canticles.txt',
 'tlg0527.tlg032': '32.Job.txt',
 'tlg0527.tlg

In [33]:
def get_lxx_lemsents(doc_id, lemmatized_sentences):
    if doc_id in lxx_mapping_dict.keys():
        lemmatized_sentences = books_sents[lxx_mapping_dict[doc_id]]
        lemmata_source_dict[doc_id] = "lxxmorph"
    return lemmatized_sentences

LAGT["lemmatized_sentences"] = LAGT.apply(lambda row: get_lxx_lemsents(row["doc_id"], row["lemmatized_sentences"]), axis=1)

In [34]:
LAGT["lemmatized_sentences"].notnull().sum()

82

# GLAUx

In [35]:
%%time
### GET PATHS+NAMES OF ALL FILES OF INTEREST WITHIN A GITHUB REPO
repo = g.get_user("perseids-publications").get_repo("glaux-trees" )
paths = []
contents = repo.get_contents("public/xml")
while contents:
    file_content = contents.pop(0)
    path_and_name = file_content.path
    if ".xml" in str(path_and_name):
        paths.append(path_and_name)

CPU times: user 15.1 ms, sys: 2.9 ms, total: 18 ms
Wall time: 1.62 s


In [36]:
len(paths)

936

In [37]:
paths[0]

'public/xml/0001-001.xml'

In [38]:
#test
response = requests.get('https://raw.githubusercontent.com/perseids-publications/glaux-trees/master/' + paths[0])
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify()[:1000])

<treebank direction="ltr" format="pedalion" version="2" xml:lang="grc" xmlns:saxon="http://saxon.sf.net/">
 <sentence document_id="0001-001" id="1" subdoc="1">
  <word form="ἀρχόμενος" head="640963" id="640955" lemma="ἄρχω" line="1" postag="v-sppemn-" relation="ADV">
  </word>
  <word form="σέο" head="640955" id="640956" lemma="σύ" line="1" postag="p-s---cg-" relation="OBJ">
  </word>
  <word form="," head="640955" id="640957" lemma="," line="1" postag="u--------" relation="PUNCT">
  </word>
  <word form="Φοῖβε" head="640963" id="640958" lemma="Φοῖβος" line="1" postag="n-s---mv-" relation="ExD">
  </word>
  <word form="," head="640955" id="640959" lemma="," line="1" postag="u--------" relation="PUNCT">
  </word>
  <word form="παλαιγενέων" head="640962" id="640960" lemma="παλαιγενής" line="1" postag="a-p---mg-" relation="ATR">
  </word>
  <word form="κλέα" head="640963" id="640961" lemma="κλέος" line="1" postag="n-p---na-" relation="SBJ">
  </word>
  <word form="φωτῶν" head="640961" id=

In [39]:
def from_filename_to_docid(filename):
    filename = filename.rpartition("/")[2]
    parts = re.split("\-|\.", filename)[:2]
    docid = "tlg"+parts[0] + ".tlg" + parts[1]
    return docid
glaux_docids = [from_filename_to_docid(filename) for filename in paths]
glaux_docids[:10]

['tlg0001.tlg001',
 'tlg0002.tlg004',
 'tlg0003.tlg001',
 'tlg0003.tlg002',
 'tlg0005.tlg001',
 'tlg0005.tlg002',
 'tlg0005.tlg005',
 'tlg0006.tlg007',
 'tlg0006.tlg008',
 'tlg0006.tlg009']

In [40]:
workid_filename_dict = dict([(from_filename_to_docid(filename), filename)  for filename in paths])

In [41]:
len(LAGT)

1818

In [42]:
len(set(LAGT["doc_id"]) & set(glaux_docids))

862

### GLAUx simple test with one work

In [43]:
base_url = "https://raw.githubusercontent.com/perseids-publications/glaux-trees/master/"

In [44]:
response = requests.get(base_url + paths[0])
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify()[:5000])

<treebank direction="ltr" format="pedalion" version="2" xml:lang="grc" xmlns:saxon="http://saxon.sf.net/">
 <sentence document_id="0001-001" id="1" subdoc="1">
  <word form="ἀρχόμενος" head="640963" id="640955" lemma="ἄρχω" line="1" postag="v-sppemn-" relation="ADV">
  </word>
  <word form="σέο" head="640955" id="640956" lemma="σύ" line="1" postag="p-s---cg-" relation="OBJ">
  </word>
  <word form="," head="640955" id="640957" lemma="," line="1" postag="u--------" relation="PUNCT">
  </word>
  <word form="Φοῖβε" head="640963" id="640958" lemma="Φοῖβος" line="1" postag="n-s---mv-" relation="ExD">
  </word>
  <word form="," head="640955" id="640959" lemma="," line="1" postag="u--------" relation="PUNCT">
  </word>
  <word form="παλαιγενέων" head="640962" id="640960" lemma="παλαιγενής" line="1" postag="a-p---mg-" relation="ATR">
  </word>
  <word form="κλέα" head="640963" id="640961" lemma="κλέος" line="1" postag="n-p---na-" relation="SBJ">
  </word>
  <word form="φωτῶν" head="640961" id=

In [45]:
#to obtain ordinary sentence data... (not used here)
sentences = []
for sent in soup.find_all("sentence"):
    sentence = ""
    for w in sent.find_all("word"):
        wform = w["form"]
        if w["relation"]=="PUNCT":
            sentence += wform
        else:
            sentence += " " + wform
    sentences.append(sentence[1:])
sentences

['ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ.',
 'τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδ’ ἀνέρος, ὅντιν’ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπ’ ἐννεσίῃσι δαμῆναι.',
 'δηρὸν δ’ οὐ μετέπειτα τεὴν κατὰ βάξιν Ἰήσων χειμερίοιο ῥέεθρα κιὼν διὰ ποσσὶν Ἀναύρου ἄλλο μὲν ἐξεσάωσεν ὑπ’ ἰλύος, ἄλλο δ’ ἔνερθεν κάλλιπεν αὖθι πέδιλον ἐνισχόμενον προχοῇσιν.',
 'ἵκετο δ’ ἐς Πελίην αὐτοσχεδὸν ἀντιβολήσων εἰλαπίνης, ἣν πατρὶ Ποσειδάωνι καὶ ἄλλοις ῥέζε θεοῖς, Ἥρης δὲ Πελασγίδος οὐκ ἀλέγιζεν.',
 'αἶψα δὲ τόνγ’ ἐσιδὼν ἐφράσσατο, καί οἱ ἄεθλον ἔντυε ναυτιλίης πολυκηδέος, ὄφρ’ ἐνὶ πόντῳ ἠὲ καὶ ἀλλοδαποῖσι μετ’ ἀνδράσι νόστον ὀλέσσῃ.',
 'νῆα μὲν οὖν οἱ πρόσθεν ἐπικλείουσιν ἀοιδοὶ Ἄργον Ἀθηναίης καμέειν ὑποθημοσύνῃσιν.',
 'νῦν δ’ ἂν ἐγὼ γενεήν τε καὶ οὔνομα μυθησαίμην ἡρώων, δολιχῆς τε πόρους ἁλός, ὅσσα τ’ ἔρεξαν πλαζόμενοι·',
 'Μοῦσαι δ’ ὑποφήτορες εἶεν ἀοιδῆς.',
 'π

In [46]:
paths[0]

'public/xml/0001-001.xml'

In [47]:
workid_filename_dict

{'tlg0001.tlg001': 'public/xml/0001-001.xml',
 'tlg0002.tlg004': 'public/xml/0002-004.xml',
 'tlg0003.tlg001': 'public/xml/0003-001.xml',
 'tlg0003.tlg002': 'public/xml/0003-002.xml',
 'tlg0005.tlg001': 'public/xml/0005-001.xml',
 'tlg0005.tlg002': 'public/xml/0005-002.xml',
 'tlg0005.tlg005': 'public/xml/0005-005.xml',
 'tlg0006.tlg007': 'public/xml/0006-007.xml',
 'tlg0006.tlg008': 'public/xml/0006-008.xml',
 'tlg0006.tlg009': 'public/xml/0006-009.xml',
 'tlg0006.tlg010': 'public/xml/0006-010.xml',
 'tlg0006.tlg011': 'public/xml/0006-011.xml',
 'tlg0006.tlg012': 'public/xml/0006-012.xml',
 'tlg0006.tlg013': 'public/xml/0006-013.xml',
 'tlg0006.tlg014': 'public/xml/0006-014.xml',
 'tlg0006.tlg015': 'public/xml/0006-015.xml',
 'tlg0006.tlg016': 'public/xml/0006-016.xml',
 'tlg0006.tlg017': 'public/xml/0006-017.xml',
 'tlg0006.tlg018': 'public/xml/0006-018.xml',
 'tlg0006.tlg019': 'public/xml/0006-019.xml',
 'tlg0006.tlg031': 'public/xml/0006-031.xml',
 'tlg0007.tlg001': 'public/xml/000

In [48]:
def lemmata_from_glaux(workid, lemmatized_sentences):
    if workid in workid_filename_dict.keys():
        lemmata_source_dict = "glaux"
        fn = workid_filename_dict[workid]
        response = requests.get(base_url + fn)
        soup = BeautifulSoup(response.text, "html.parser")
        lemmatized_sentences = []
        for sent in soup.find_all("sentence"):
            sentence_lemmata = []
            for w in sent.find_all("word"):
                try:
                    if w["postag"][0] in ["n", "v", "a"]:
                        sentence_lemmata.append(w["lemma"])
                except:
                    try:
                        sentence_lemmata.append(w["lemma"])
                    except:
                        pass

            lemmatized_sentences.append(sentence_lemmata)
    return lemmatized_sentences
LAGT["lemmatized_sentences"] = LAGT.apply(lambda row: lemmata_from_glaux(row["doc_id"], row["lemmatized_sentences"]), axis=1)

In [49]:
LAGT["lemmatized_sentences"].notnull().sum()

944

In [50]:
for doc_id in LAGT[LAGT["lemmatized_sentences"].notnull()]["doc_id"]:
    if doc_id not in lemmata_source_dict.keys():
        lemmata_source_dict[doc_id] = "glaux"

# Pedalion trees data

In [51]:
%%time
### GET PATHS+NAMES OF ALL FILES OF INTEREST WITHIN A GITHUB REPO
repo = g.get_user("perseids-publications").get_repo("pedalion-trees" )
paths = []
contents = repo.get_contents("public/xml")
while contents:
    file_content = contents.pop(0)
    path_and_name = file_content.path
    if ".xml" in str(path_and_name):
        paths.append(path_and_name)

CPU times: user 14.8 ms, sys: 3.03 ms, total: 17.8 ms
Wall time: 1.32 s


In [52]:
paths[0]

'public/xml/achar.xml'

In [53]:
base_url = "https://raw.githubusercontent.com/perseids-publications/pedalion-trees/master/"

In [54]:
def lemmatized_sentences_by_path(path, base_url=base_url):
    response = requests.get(base_url + path)
    soup = BeautifulSoup(response.text, "html.parser")
    lemmata_sentences = []
    doc_ids = []
    for sent in soup.find_all("sentence"):
        doc_ids.append(sent['document_id'])
        sentence_lemmata = []
        for w in sent.find_all("word"):
            try:
                if w["postag"][0] in ["n", "v", "a"]:
                    sentence_lemmata.append(w["lemma"])
            except:
                try:
                    sentence_lemmata.append(w["lemma"])
                except:
                    pass
        lemmata_sentences.append(sentence_lemmata)
    return doc_ids, lemmata_sentences

In [55]:
docs_ids, docs_lemmatized_sentences = [], []
for path in paths:
    doc_ids, lemmatized_sentences = lemmatized_sentences_by_path(path, base_url)
    docs_ids.append(doc_ids)
    docs_lemmatized_sentences.append(lemmatized_sentences)

In [56]:
docs_lemmatized_sentences[10][:10]

[['ὀφείλω',
  'Ἀργώ',
  'διαπέταμαι',
  'σκάφος',
  'Κόλχος',
  'αἶα',
  'κυάνεος',
  'Συμπληγάς',
  'νάπη',
  'Πήλιον',
  'πίπτω',
  'τέμνω',
  'πεύκη',
  'ἐρετμόω',
  'χήρ',
  'ἀνήρ',
  'ἀριστεύς',
  'παγχρύσεος',
  'δέρος',
  'Πελίας',
  'μετέρχομαι'],
 ['δέσποινα',
  'ἐμός',
  'Μήδεια',
  'πύργος',
  'γῆ',
  'πλέω',
  'Ἰώλκιος',
  'ἔρως',
  'θυμός',
  'ἐκπλήσσω',
  'Ἰάσων'],
 ['κτείνω',
  'πείθω',
  'Πελιάς',
  'κόρη',
  'πατήρ',
  'κατοικέω',
  'γῆ',
  'Κορίνθιος',
  'φίλος',
  'ἀμπλακεῖν',
  'πάτρα'],
 ['ἔχω',
  'μεμπτός',
  'βίος',
  'ἀνήρ',
  'τέκνον',
  'ἁνδάνω',
  'φυγάς',
  'πολίτης',
  'ἀφικνέομαι',
  'χθών',
  'πᾶς',
  'συμφέρω',
  'Ἰάσων'],
 ['μέγας', 'γίγνομαι', 'σωτηρία', 'γυνή', 'ἀνήρ', 'διχοστατέω'],
 ['ἐχθρός', 'πᾶς', 'νοσέω', 'φίλος'],
 ['προδίδωμι',
  'τέκνον',
  'δεσπότις',
  'ἐμός',
  'γάμος',
  'Ἰάσων',
  'βασιλικός',
  'εὐνάζω',
  'γαμέω',
  'Κρέων',
  'παῖς',
  'αἰσυμνάω',
  'χθών'],
 ['Μήδεια',
  'δύστηνος',
  'ἀτιμάζω',
  'βοάω',
  'ὅρκος',
  'ἀνακαλέω',
  '

In [57]:
ids_sents_dict = {}
for ids, sents in zip(docs_ids, docs_lemmatized_sentences):
    if len(set(ids))==1:
        ids_sents_dict[ids[0]] = sents

In [58]:
len(ids_sents_dict)

33

In [59]:
ids_sents_dict.keys()

dict_keys(['0019-001', '0058-001', '1220-001', '0554-001', '0041-001', '0059-003', '0557-001', '0537-012', '0006-003', '0343-001', 'Genesis', 'Genesis_deel2', 'Genesis_deel3', '0559-002', '0010-020', '2003-001', '0561-001', '0062-014', '0062-020', '0062-015', '0540-024', '0541-007', 'PaeanBrev', '0585-001', '4029-001', 'Ps-Luc', '0059-00x', '0260-x', '0544-002', '0005-xx', '0093-009', '0019-008', '0032-002'])

In [60]:
def add_pedalion_lemmata(doc_id, lemmatized_sentences):
    if lemmatized_sentences == None:
        doc_id = doc_id.replace("tlg", "").replace(".", "-")
        if doc_id in ids_sents_dict.keys():
            lemmatized_sentences = ids_sents_dict[doc_id]
            lemmata_source_dict[doc_id] = "pedalion"
    return lemmatized_sentences
LAGT["lemmatized_sentences"] = LAGT.apply(lambda row: add_pedalion_lemmata(row["doc_id"], row["lemmatized_sentences"]), axis=1)

In [61]:
LAGT["lemmatized_sentences"].notnull().sum()

951

# Add Gorman treebanks data

In [62]:
%%time
### GET PATHS+NAMES OF ALL FILES OF INTEREST WITHIN A GITHUB REPO
repo = g.get_user("perseids-publications").get_repo("gorman-trees" )
paths = []
contents = repo.get_contents("public/xml")
while contents:
    file_content = contents.pop(0)
    path_and_name = file_content.path
    if ".xml" in str(path_and_name):
        paths.append(path_and_name)

CPU times: user 21.2 ms, sys: 1.95 ms, total: 23.1 ms
Wall time: 1.29 s


In [63]:
paths

['public/xml/aeschines-1-1-50-bu1.xml',
 'public/xml/aeschines-1-101-150-bu1.xml',
 'public/xml/aeschines-1-151-196-bu1.xml',
 'public/xml/aeschines-1-51-100-bu1.xml',
 'public/xml/antiphon-1-bu2.xml',
 'public/xml/antiphon-2-bu2.xml',
 'public/xml/antiphon-5-bu2.xml',
 'public/xml/antiphon-6-bu2.xml',
 'public/xml/appian-bc-1-0-1-4-bu1.xml',
 'public/xml/appian-bc-1-11-14-bu1.xml',
 'public/xml/appian-bc-1-5-7-bu1.xml',
 'public/xml/appian-bc-1-8-10-bu1.xml',
 'public/xml/aristotle-politics-book-1-bu1.xml',
 'public/xml/aristotle-politics-book-2-bu2.xml',
 'public/xml/athen12-1-9-2019.xml',
 'public/xml/athen12-10-19-2019.xml',
 'public/xml/athen12-20-29-2019.xml',
 'public/xml/athen12-20-29-jan-15.xml',
 'public/xml/athen12-30-39-2019.xml',
 'public/xml/athen12-30-39-jan-15.xml',
 'public/xml/athen12-40-49-2019.xml',
 'public/xml/athen12-40-49-jan-15.xml',
 'public/xml/athen12-50-59-2019.xml',
 'public/xml/athen12-50-59-jan-15.xml',
 'public/xml/athen12-60-69-2019.xml',
 'public/xml/

In [64]:
base_url = "https://raw.githubusercontent.com/perseids-publications/gorman-trees/master/"

def lemmatized_sentences_by_path(path, base_url=base_url):
    response = requests.get(base_url + path)
    soup = BeautifulSoup(response.text, "html.parser")
    lemmata_sentences = []
    doc_paths = []
    doc_subdocs = []
    doc_ids = []
    for sent in soup.find_all("sentence"):
        doc_paths.append(path)
        doc_ids.append(sent['document_id'])
        doc_subdocs.append(sent['id'])
        sentence_lemmata = []
        for w in sent.find_all("word"):
            try:
                if w["postag"][0] in ["n", "v", "a"]:
                    sentence_lemmata.append(w["lemma"])
            except:
                try:
                    sentence_lemmata.append(w["lemma"])
                except:
                    pass
        lemmata_sentences.append(sentence_lemmata)
    return (doc_paths, doc_ids, doc_subdocs, lemmata_sentences)

In [65]:
docs_paths, docs_ids, docs_subdocs, docs_lemmatized_sentences = [], [], [], []

for path in paths:
    doc_paths, doc_ids, doc_subdocs, lemmatized_sentences = lemmatized_sentences_by_path(path, base_url)
    docs_paths.extend(doc_paths)
    docs_ids.extend(doc_ids)
    docs_subdocs.extend(doc_subdocs)
    docs_lemmatized_sentences.extend(lemmatized_sentences)

In [66]:
gorman_df = pd.DataFrame([docs_paths, docs_ids, docs_subdocs, docs_lemmatized_sentences]).T
gorman_df.head(5)

,0,1,2,3
0,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,1,"[πολίτης, ἀνήρ, Ἀθηναῖος, γραφής, γράφω, εὔθυν..."
1,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,2,"[οἶδα, ἔνοχος, εἰμί, ὀλίγος, πρότερος, ἀκούω, ..."
2,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,3,"[ἔοικα, ἀνήρ, Ἀθηναῖος, ἔθω, λόγος, λέγω3, δημ..."
3,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,4,"[ἴδιος, ἔχθρα, πολύς, κοινός, ἐπανορθόω]"
4,public/xml/aeschines-1-1-50-bu1.xml,urn:cts:greekLit:tlg0026.tlg001.perseus-grc1,5,"[ὅλος, ἀγών, φαίνω, πόλις, αἴτιος, εἰμί, Τίμαρ..."


In [67]:
gorman_df.columns = ["path_raw", "doc_id_raw", "subdoc", "sentence"]

In [68]:
gorman_df["path"] = gorman_df["path_raw"].apply(lambda x: x.rpartition("/")[2])

In [69]:
gorman_df["doc_id_raw"].unique().tolist()

['urn:cts:greekLit:tlg0026.tlg001.perseus-grc1',
 'http://perseids.org/annotsrc/urn:cts:greekLit:tlg0028.tlg001.perseus-grc1',
 'urn:cts:greekLit:tlg0028.tlg002.perseus-grc1',
 'urn:cts:greekLit:tlg0028.tlg005.perseus-grc1',
 'urn:cts:greekLit:tlg0028.tlg006.perseus-grc1',
 'urn:cts:greekLit:tlg0551.tlg017.perseus-grc2',
 'urn:cts:greekLit:tlg0086.tlg035.perseus-grc1:1',
 'urn:cts:greekLit:tlg0086.tlg035.perseus-grc1:2',
 'urn:cts:greekLit:tlg0008.tlg001.perseus-grc1',
 'Perseus:text:2008.01.0405',
 'urn:cts:greekLit:tlg0014.tlg059.perseus-grc2',
 '0014-059',
 '0014-001',
 'urn:cts:greekLit:tlg0014.tlg018.perseus-grc1',
 'urn:cts:greekLit:tlg0014.tlg004.perseus-grc1',
 '0014-046',
 '0014-047',
 '0014-049',
 '0014-050',
 '0014-052',
 '0014-053',
 'http://perseids.org/annotsrc/urn:cts:greekLit:tlg0060.tlg001.perseus-grc3',
 'urn:cts:greekLit:tlg0060.tlg001.perseus-grc3',
 'urn:cts:greekLit:tlg0081.tlg001.perseus-grc1',
 'urn:cts:greekLit:tlg0016.tlg001.perseus-grc1',
 'urn:cts:greekLit:t

In [70]:
def find_match(x):
    doc_id = None
    match = re.search("tlg\d{4}\.tlg\d{3}", x)
    if match:
        doc_id = match.group()
    else:
        match = re.search("(\d{4})\-(\d{3})", x)
        if match:
            doc_id = "tlg" + match.groups()[0] + ".tlg" + match.groups()[1]
    return doc_id

gorman_df["doc_id"] = gorman_df["doc_id_raw"].apply(find_match)

In [71]:
gorman_df.sort_values("path")["path"].unique()

array(['aeschines-1-1-50-bu1.xml', 'aeschines-1-101-150-bu1.xml',
       'aeschines-1-151-196-bu1.xml', 'aeschines-1-51-100-bu1.xml',
       'antiphon-1-bu2.xml', 'antiphon-2-bu2.xml', 'antiphon-5-bu2.xml',
       'antiphon-6-bu2.xml', 'appian-bc-1-0-1-4-bu1.xml',
       'appian-bc-1-11-14-bu1.xml', 'appian-bc-1-5-7-bu1.xml',
       'appian-bc-1-8-10-bu1.xml', 'aristotle-politics-book-1-bu1.xml',
       'aristotle-politics-book-2-bu2.xml', 'athen12-1-9-2019.xml',
       'athen12-10-19-2019.xml', 'athen12-20-29-2019.xml',
       'athen12-20-29-jan-15.xml', 'athen12-30-39-2019.xml',
       'athen12-30-39-jan-15.xml', 'athen12-40-49-2019.xml',
       'athen12-40-49-jan-15.xml', 'athen12-50-59-2019.xml',
       'athen12-50-59-jan-15.xml', 'athen12-60-69-2019.xml',
       'athen12-60-69-jan-15.xml', 'athen12-70-81-2019.xml',
       'athen12-70-81-jan-15.xml', 'athen13-1-9-2019.xml',
       'athen13-1-9-jan-15.xml', 'athen13-10-19-2019.xml',
       'athen13-10-19-jan-15.xml', 'athen13-20-29-

In [72]:
docs_paths_dict = {}
for doc_id in gorman_df["doc_id"].unique():
    docs_paths_dict[doc_id] = gorman_df[gorman_df["doc_id"]==doc_id]["path"].unique().tolist()

In [73]:
for_manual_sorting = ""
for key, val in docs_paths_dict.items():
    
    if len(val)>1:
        for_manual_sorting += "\ndocs_paths_dict[\"" + key + "\"] = " + str(sorted(val))

In [74]:
print(for_manual_sorting)


docs_paths_dict["tlg0026.tlg001"] = ['aeschines-1-1-50-bu1.xml', 'aeschines-1-101-150-bu1.xml', 'aeschines-1-151-196-bu1.xml', 'aeschines-1-51-100-bu1.xml']
docs_paths_dict["tlg0551.tlg017"] = ['appian-bc-1-0-1-4-bu1.xml', 'appian-bc-1-11-14-bu1.xml', 'appian-bc-1-5-7-bu1.xml', 'appian-bc-1-8-10-bu1.xml']
docs_paths_dict["tlg0086.tlg035"] = ['aristotle-politics-book-1-bu1.xml', 'aristotle-politics-book-2-bu2.xml']
docs_paths_dict["tlg0008.tlg001"] = ['athen12-1-9-2019.xml', 'athen12-10-19-2019.xml', 'athen12-20-29-2019.xml', 'athen12-20-29-jan-15.xml', 'athen12-30-39-2019.xml', 'athen12-30-39-jan-15.xml', 'athen12-40-49-2019.xml', 'athen12-40-49-jan-15.xml', 'athen12-50-59-2019.xml', 'athen12-50-59-jan-15.xml', 'athen12-60-69-2019.xml', 'athen12-60-69-jan-15.xml', 'athen12-70-81-2019.xml', 'athen12-70-81-jan-15.xml', 'athen13-1-9-2019.xml', 'athen13-1-9-jan-15.xml', 'athen13-10-19-2019.xml', 'athen13-10-19-jan-15.xml', 'athen13-20-29-2019.xml', 'athen13-20-29-jan-15.xml', 'athen13-30-

In [75]:
docs_paths_dict["tlg0026.tlg001"] = ['aeschines-1-1-50-bu1.xml', 'aeschines-1-51-100-bu1.xml', 'aeschines-1-101-150-bu1.xml', 'aeschines-1-151-196-bu1.xml']
docs_paths_dict["tlg0551.tlg017"] = ['appian-bc-1-0-1-4-bu1.xml', 'appian-bc-1-5-7-bu1.xml', 'appian-bc-1-8-10-bu1.xml', 'appian-bc-1-11-14-bu1.xml']
docs_paths_dict["tlg0086.tlg035"] = ['aristotle-politics-book-1-bu1.xml', 'aristotle-politics-book-2-bu2.xml']
docs_paths_dict["tlg0008.tlg001"] = ['athen12-1-9-2019.xml', 'athen12-10-19-2019.xml', 'athen12-20-29-2019.xml', 'athen12-30-39-2019.xml', 'athen12-40-49-2019.xml', 'athen12-50-59-2019.xml', 'athen12-60-69-2019.xml', 'athen12-70-81-2019.xml', 'athen13-1-9-2019.xml', 'athen13-10-19-2019.xml', 'athen13-20-29-2019.xml', 'athen13-30-39-2019.xml',  'athen13-40-49-2019.xml', 'athen13-50-59-2019.xml', 'athen13-60-69-2019.xml', 'athen13-70-79-2019.xml',  'athen13-80-89-2019.xml', 'athen13-90-95-2019.xml']
docs_paths_dict["tlg0014.tlg018"] = ['demosthenes-18-1-50-bu2.xml', 'demosthenes-18-51-100-bu1.xml', 'demosthenes-18-101-150-bu2.xml', 'demosthenes-18-151-200-bu2.xml', 'demosthenes-18-201-275-bu1.xml', 'demosthenes-18-276-324-bu1.xml']
docs_paths_dict["tlg0060.tlg001"] = ['diodsic-11-1-20-bu4.xml', 'diodsic11-21-40-bu2.xml', 'diodsic11-41-60-bu1.xml', 'diodsic11-61-80-bu1.xml', 'diodsic-11-81-92-bu1.xml']
docs_paths_dict["tlg0081.tlg001"] = ['dion-hal-1-1-15-bu2.xml', 'dion-hal-1-16-30-bu1.xml', 'dion-hal-1-31-45-bu1.xml', 'dion-hal-1-46-60-bu1.xml', 'dion-hal-1-61-75-bu1.xml', 'dion-hal-1-76-90-bu1.xml']
docs_paths_dict["tlg0016.tlg001"] = ['hdt-1-1-19-bu3-2019.xml', 'hdt-1-20-39-bu2-2019.xml', 'hdt-1-40-59-bu2-2019.xml', 'hdt-1-60-79-bu2-2019.xml', 'hdt-1-80-99-bu5-2019.xml', 'hdt-1-100-119-bu3-2019.xml', 'hdt-1-120-149-bu2-2019.xml', 'hdt-1-150-169-bu3-2019.xml', 'hdt-1-170-189-bu2-2019.xml', 'hdt-1-190-216-bu2-2019.xml']
docs_paths_dict["tlg0526.tlg004"] = ['josephus-bj-1-1-2-bu1.xml',  'josephus-bj-1-3-5-bu2.xml', 'josephus-bj-1-6-10-bu1.xml', 'josephus-bj-1-11-15-bu1.xml', 'josephus-bj-1-16-20-bu1.xml', 'josephus-bj-1-21-25-bu1.xml']
docs_paths_dict["tlg0007.tlg015"] = ['plut-alcib-1-17-bu1.xml', 'plut-alcib-18-39-bu1.xml']
docs_paths_dict["tlg0007.tlg004"] = ['plutarch-lycurgus-1-15-bu4.xml', 'plutarch-lycurgus-16-31-bu2.xml']
#docs_paths_dict["tlg0543.tlg001"] = ['polybius1-1-9-2017.xml', 'polybius1-10-19-2017.xml', 'polybius1-20-29-2017.xml', 'polybius1-30-39-2017.xml', 'polybius1-40-49-2017.xml', 'polybius1-50-59-2017.xml', 'polybius1-60-69-2017.xml', 'polybius1-70-79-2017.xml', 'polybius1-80-88-2017.xml', 'polybius-2-1-10-bu1.xml', 'polybius-2-11-20-bu1.xml', 'polybius-2-21-30-bu2.xml', 'polybius-2-31-40-bu2.xml', 'polybius-2-41-50-bu1.xml', 'polybius-2-51-60-bu1.xml', 'polybius-2-61-71-bu2.xml', 'polybius-6-2-15-bu1.xml', 'polybius-6-16-30-bu1.xml', 'polybius-6-31-45-bu1.xml', 'polybius-6-46-58-bu1.xml', 'polybius-9-1-20-bu1.xml', 'polybius-9-21-33-bu1.xml', 'polybius-9-34-45-bu1.xml','polybius-10-1-10-bu1.xml', 'polybius-10-11-20-bu1.xml', 'polybius-10-21-35-bu2.xml', 'polybius-10-36-49-bu1.xml' 'polybius-21-1-10-bu1.xml', 'polybius-21-11-20-bu1.xml', 'polybius-21-21-30-bu1.xml', 'polybius-21-31-47-bu1.xml']
#docs_paths_dict["tlg0003.tlg001"] = ['thuc-1-1-20-bu5.xml', 'thuc-1-21-40-bu4.xml', 'thuc-1-41-60-bu3.xml', 'thuc-1-61-80-bu3.xml', 'thuc-1-81-100-bu2.xml','thuc-1-101-120-bu2.xml', 'thuc-1-121-146-bu3.xml' 'thuc-3-1-20-bu1.xml', 'thuc-3-21-40-bu1.xml']
docs_paths_dict["tlg0032.tlg007"] = ['xen-cyr-1-1-2-bu1.xml', 'xen-cyr-1-3-4-bu1.xml', 'xen-cyr-1-5-bu1.xml', 'xen-cyr-1-6-bu1.xml', 'xen-cyr-7-1-3-tree.xml', 'xen-cyr-7-4-5-tree.xml', 'xen-cyr-8-1-8-4-bu1.xml', 'xen-cyr-8-5-7-bu1.xml', 'xen-cyr-8-8-bu1.xml']
docs_paths_dict["tlg0032.tlg001"] = ['xen-hell-1-1-4-bu2.xml', 'xen-hell-1-5-7-bu1.xml', 'xen-hell-2-bu1.xml', 'xen-hell-3-bu1.xml']

In [76]:
def get_gorman_sentences(doc_id, lemmatized_sentences):
    if lemmatized_sentences == None:
        if doc_id in docs_paths_dict.keys():
            key = doc_id
            lemmatized_sentences = []
            subset = gorman_df[gorman_df["doc_id"]==key]
            for path in docs_paths_dict[key]:
                lemmatized_sentences.extend(subset[subset["path"]==path].sort_values("subdoc")["sentence"].tolist())
            lemmata_source_dict[doc_id] = "gorman"
    return lemmatized_sentences

LAGT["lemmatized_sentences"] = LAGT.apply(lambda row: get_gorman_sentences(row["doc_id"], row["lemmatized_sentences"]), axis=1)

In [77]:
LAGT["lemmatized_sentences"].notnull().sum()

953

### AGDT

In [302]:
!mkdir ../data/large_files/AGDT
!curl -L https://github.com/PerseusDL/treebank_data/zipball/master -o ../data/large_files/AGDt/agdt.zip
!unzip ../data/large_files/AGDT/agdt.zip -d ../data/large_files/AGDT

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 40.9M    0 40.9M    0     0   506k      0 --:--:--  0:01:22 --:--:-- 3365k7 --:--:-- 28316    0 --:--:--  0:00:28 --:--:--  6637--:--:--  0:01:15 --:--:-- 2995k
Archive:  ../data/large_files/AGDT/agdt.zip
92de910336e5ea20e9acafbd82be9ad937d53000
   creating: ../data/large_files/AGDT/PerseusDL-treebank_data-92de910/
   creating: ../data/large_files/AGDT/PerseusDL-treebank_data-92de910/AGDT2/
   creating: ../data/large_files/AGDT/PerseusDL-treebank_data-92de910/AGDT2/corpus/
  inflating: ../data/large_files/AGDT/PerseusDL-treebank_data-92de910/AGDT2/corpus/liveannotations.md  
  inflating: ../data/large_files/AGDT/PerseusDL-treebank_data-92de910/AGDT2/corpus/readme.md  
   creating: ../data/large_files/AGDT/PerseusDL-treebank_data-92de910/AGDT2/corpus

In [78]:
path2_1 = "../data/large_files/AGDT/PerseusDL-treebank_data-92de910/v2.1/Greek/texts/"
agdt_paths = [path2_1 + f for f in os.listdir(path2_1)]
doc_ids = [re.search("(.+\d{3})(.+)", f).groups()[0]  for f in os.listdir(path2_1)]
agdt_paths[:3]

['../data/large_files/AGDT/PerseusDL-treebank_data-92de910/v2.1/Greek/texts/tlg0085.tlg004.perseus-grc2.tb.xml',
 '../data/large_files/AGDT/PerseusDL-treebank_data-92de910/v2.1/Greek/texts/tlg0013.tlg002.perseus-grc1.tb.xml',
 '../data/large_files/AGDT/PerseusDL-treebank_data-92de910/v2.1/Greek/texts/tlg0059.tlg001.perseus-grc1.tb.xml']

In [79]:
agdt_dict = dict(zip(doc_ids, agdt_paths))

In [80]:
soup = BeautifulSoup(open(agdt_paths[0], 'r').read(), "html.parser")

In [81]:
lemmata_sentences = []
for sent in soup.find_all("sentence"):
    sentence_lemmata = []
    for w in sent.find_all("word"):
        try:
            if w["postag"][0] in ["n", "v", "a"]:
                sentence_lemmata.append(w["lemma"])
        except:
            try:
                sentence_lemmata.append(w["lemma"])
            except:
                pass
    lemmata_sentences.append(sentence_lemmata)

In [82]:
lemmata_sentences[:3]

[['Κάδμος',
  'πολίτης',
  'χρή',
  'λέγω',
  'καίριος',
  'φυλάσσω',
  'πρᾶγος',
  'πρύμνα',
  'πόλις',
  'οἴαξ',
  'νωμάω',
  'βλέφαρον',
  'κοιμάω',
  'ὕπνος'],
 ['πράσσω', 'αἰτία', 'θεός'],
 ['γίγνομαι',
  'συμφορά',
  'τυγχάνω',
  'Ἐτεοκλῆς',
  'εἷς',
  'πολύς',
  'πόλις',
  'ὑμνέω',
  'ἀστός',
  'φροίμιον',
  'πολύρροθος',
  'οἴμωγμα',
  'Ζεύς',
  'ἀλεξητήριος',
  'ἐπώνυμος',
  'γίγνομαι',
  'Καδμεῖος',
  'πόλις']]

In [83]:
agdt_uncomplete = ["tlg0096.tlg002",
                   "tlg0008.tlg001",
                   "tlg0060.tlg001",
                   "tlg0016.tlg001",
                   "tlg0543.tlg001",
                   "tlg0548.tlg001",
                   "tlg0003.tlg001"
                   ]

In [84]:
def get_agdt_sentences(doc_id, lemmatized_sentences):
    if doc_id in agdt_dict.keys():
        if doc_id not in agdt_uncomplete:
            soup = BeautifulSoup(open(agdt_dict[doc_id], 'r').read(), "html.parser")
            lemmatized_sentences = []
            for sent in soup.find_all("sentence"):
                sentence_lemmata = []
                for w in sent.find_all("word"):
                    try:
                        if w["postag"][0] in ["n", "v", "a"]:
                            sentence_lemmata.append(w["lemma"])
                    except:
                        try:
                            sentence_lemmata.append(w["lemma"])
                        except:
                            pass
                lemmatized_sentences.append(sentence_lemmata)
            lemmata_source_dict[doc_id] = "agdt"
    return lemmatized_sentences

LAGT["lemmatized_sentences"] = LAGT.apply(lambda row: get_agdt_sentences(row["doc_id"], row["lemmatized_sentences"]), axis=1)

In [85]:
LAGT[LAGT["doc_id"].isin(doc_ids)]

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences
326,tlg0096,tlg0096.tlg002,tlg0096.tlg002.opp-grc3.xml,Aesop,Fabulae,Ὑπὸ τῶν κακῶν τὰ ἀγαθὰ ἐδιώχθη ὡς ἀσθενῆ ὄντα·...,29505,1Kgr,None
1158,tlg0003,tlg0003.tlg001,tlg0003.tlg001.perseus-grc2.xml,Thucydides,Ἱστορίαι,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν\...,150363,perseus,"[[Θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
1184,tlg0007,tlg0007.tlg004,tlg0007.tlg004.perseus-grc2.xml,Plutarch,Λυκοῦργος,περὶ Λυκούργου τοῦ νομοθέτου καθόλου μὲν οὐδὲ...,9656,perseus,"[[Λυκοῦργος, νομοθέτης, εἰμί, εἶπον, ἀναμφισβή..."
1195,tlg0007,tlg0007.tlg015,tlg0007.tlg015.perseus-grc2.xml,Plutarch,Ἀλκιβιάδης,τὸ Ἀλκιβιάδου γένος ἄνωθεν Εὐρυσάκην τὸν Αἴαν...,10239,perseus,"[[Ἀλκιβιάδης, γένος, Εὐρυσάκης, Αἴας, ἀρχηγός,..."
1336,tlg0008,tlg0008.tlg001,tlg0008.tlg001.perseus-grc4.xml,Athenaeus,Δειπνοσοφισταί,\n\nἈθήναιος μὲν ὁ τῆς βίβλου πατήρ· ποιεῖται ...,327324,perseus,"[[Ἀθηναῖος, βίβλος, πατήρ], [ποιέω, λόγος, Τιμ..."
1367,tlg0011,tlg0011.tlg001,tlg0011.tlg001.perseus-grc2.xml,Sophocles,Τραχίνιαι,"λόγος μέν ἐστʼ ἀρχαῖος ἀνθρώπων φανείς, ὡς οὐκ...",7294,perseus,"[[λόγος, εἰμί, ἀρχαῖος, ἄνθρωπος, φαίνω, αἰών,..."
1368,tlg0011,tlg0011.tlg002,tlg0011.tlg002.perseus-grc2.xml,Sophocles,Ἀντιγόνη,"ὦ κοινὸν αὐτάδελφον Ἰσμήνης κάρα, ἆρʼ οἶσθʼ ὅ ...",7360,perseus,"[[κοινός, αὐτάδελφος, Ἰσμήνη, κάρα, οἶδα, Ζεύς..."
1369,tlg0011,tlg0011.tlg003,tlg0011.tlg003.perseus-grc2.xml,Sophocles,Αἴας,"ἀεὶ μέν, ὦ παῖ Λαρτίου, δέδορκά σε πεῖράν τινʼ...",7915,perseus,"[[παῖς, λάρτιος, δέρκομαι, πεῖρα, ἐχθρός, ἁρπά..."
1370,tlg0011,tlg0011.tlg004,tlg0011.tlg004.perseus-grc2.xml,Sophocles,Οἰδίπους Τύραννος,"ὦ τέκνα, Κάδμου τοῦ πάλαι νέα τροφή, τίνας ποθ...",9292,perseus,"[[τέκνον, Κάδμος, νέος, τροφή, ἕδρα, θοάζω, ἱκ..."
1371,tlg0011,tlg0011.tlg005,tlg0011.tlg005.perseus-grc2.xml,Sophocles,Ἠλέκτρα,ὦ τοῦ στρατηγήσαντος ἐν Τροίᾳ ποτὲ Ἀγαμέμνονος...,8704,perseus,"[[στρατηγέω, Τροία, Ἀγαμέμνων, παῖς, ἐκεῖνος, ..."


In [ ]:
#LAGT.at[xxx, "lemmatized_sentences"]

In [86]:
lemmata_source_dict

{'tlg0031.tlg001': 'morphgnt',
 'tlg0031.tlg002': 'morphgnt',
 'tlg0031.tlg003': 'morphgnt',
 'tlg0031.tlg004': 'morphgnt',
 'tlg0031.tlg005': 'morphgnt',
 'tlg0031.tlg006': 'morphgnt',
 'tlg0031.tlg007': 'morphgnt',
 'tlg0031.tlg008': 'morphgnt',
 'tlg0031.tlg009': 'morphgnt',
 'tlg0031.tlg010': 'morphgnt',
 'tlg0031.tlg011': 'morphgnt',
 'tlg0031.tlg012': 'morphgnt',
 'tlg0031.tlg013': 'morphgnt',
 'tlg0031.tlg014': 'morphgnt',
 'tlg0031.tlg015': 'morphgnt',
 'tlg0031.tlg016': 'morphgnt',
 'tlg0031.tlg017': 'morphgnt',
 'tlg0031.tlg018': 'morphgnt',
 'tlg0031.tlg019': 'morphgnt',
 'tlg0031.tlg020': 'morphgnt',
 'tlg0031.tlg021': 'morphgnt',
 'tlg0031.tlg022': 'morphgnt',
 'tlg0031.tlg023': 'morphgnt',
 'tlg0031.tlg024': 'morphgnt',
 'tlg0031.tlg025': 'morphgnt',
 'tlg0031.tlg026': 'morphgnt',
 'tlg0031.tlg027': 'morphgnt',
 'tlg0527.tlg001': 'lxxmorph',
 'tlg0527.tlg002': 'lxxmorph',
 'tlg0527.tlg003': 'lxxmorph',
 'tlg0527.tlg004': 'lxxmorph',
 'tlg0527.tlg005': 'lxxmorph',
 'tlg052

In [89]:
key = "2003-001"
groups = re.search("(\d{4})\-(\d{3})", key).groups()
key_updated = "tlg" + groups[0] + ".tlg" + groups[1]
key_updated

'tlg2003.tlg001'

In [91]:
for key in list(lemmata_source_dict.keys()):
    if "tlg" not in key:
        groups = re.search("(\d{4})\-(\d{3})", key).groups()
        key_updated = "tlg" + groups[0] + ".tlg" + groups[1]
        lemmata_source_dict[key_updated] = lemmata_source_dict[key]
        del lemmata_source_dict[key]

In [92]:
lemmata_source_dict

{'tlg0031.tlg001': 'morphgnt',
 'tlg0031.tlg002': 'morphgnt',
 'tlg0031.tlg003': 'morphgnt',
 'tlg0031.tlg004': 'morphgnt',
 'tlg0031.tlg005': 'morphgnt',
 'tlg0031.tlg006': 'morphgnt',
 'tlg0031.tlg007': 'morphgnt',
 'tlg0031.tlg008': 'morphgnt',
 'tlg0031.tlg009': 'morphgnt',
 'tlg0031.tlg010': 'morphgnt',
 'tlg0031.tlg011': 'morphgnt',
 'tlg0031.tlg012': 'morphgnt',
 'tlg0031.tlg013': 'morphgnt',
 'tlg0031.tlg014': 'morphgnt',
 'tlg0031.tlg015': 'morphgnt',
 'tlg0031.tlg016': 'morphgnt',
 'tlg0031.tlg017': 'morphgnt',
 'tlg0031.tlg018': 'morphgnt',
 'tlg0031.tlg019': 'morphgnt',
 'tlg0031.tlg020': 'morphgnt',
 'tlg0031.tlg021': 'morphgnt',
 'tlg0031.tlg022': 'morphgnt',
 'tlg0031.tlg023': 'morphgnt',
 'tlg0031.tlg024': 'morphgnt',
 'tlg0031.tlg025': 'morphgnt',
 'tlg0031.tlg026': 'morphgnt',
 'tlg0031.tlg027': 'morphgnt',
 'tlg0527.tlg001': 'lxxmorph',
 'tlg0527.tlg002': 'lxxmorph',
 'tlg0527.tlg003': 'lxxmorph',
 'tlg0527.tlg004': 'lxxmorph',
 'tlg0527.tlg005': 'lxxmorph',
 'tlg052

In [93]:
LAGT.columns

Index(['author_id', 'doc_id', 'filename', 'author', 'title', 'string',
       'wordcount', 'source', 'lemmatized_sentences'],
      dtype='object')

In [94]:
LAGT["lemmata_source"] = LAGT["doc_id"].apply(lambda x: lemmata_source_dict[x] if x in lemmata_source_dict.keys() else None)

In [95]:
LAGT["lemmata_source"].notnull().sum()

951

In [96]:
LAGT.to_json("../data/large_files/LAGT_treebanks_20240116.json")

In [97]:
s.write_file("SDAM_data/AGT/LAGT_treebanks_20240116.json", LAGT)

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/AGT/LAGT_treebanks_20240116.json"
